In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: justinwu314 (justin-wu). Use `wandb login --relogin` to force relogin


True

In [2]:
#make sure to cd into smallcap-main
#load in datasets
import importlib
#importlib.reload(src.utils)
import torch
import sys
import argparse
import os
import pandas as pd
#sys.path.append('/home/ec2-user/smallcap-main/src')
from transformers.models.auto.configuration_auto import AutoConfig
from transformers import AutoTokenizer, CLIPFeatureExtractor, AutoModel, AutoModelForCausalLM
from transformers import Seq2SeqTrainer, default_data_collator, Seq2SeqTrainingArguments

from transformers import VisionEncoderDecoderModel, CLIPModel, CLIPVisionModel,EncoderDecoderModel
from src.vision_encoder_decoder import SmallCap, SmallCapConfig
from src.gpt2 import ThisGPT2Config, ThisGPT2LMHeadModel
from src.xglm import ThisXGLMConfig, ThisXGLMForCausalLM
from src.opt import ThisOPTConfig, ThisOPTForCausalLM

from src.utils import *
from transformers import ViTFeatureExtractor, AutoTokenizer, CLIPFeatureExtractor, AutoModel, AutoModelForCausalLM
from transformers.models.auto.configuration_auto import AutoConfig
from src.vision_encoder_decoder import SmallCap, SmallCapConfig
from src.gpt2 import ThisGPT2Config, ThisGPT2LMHeadModel
from src.utils import *
import pandas
import src.utils
import numpy as np

import csv
from pandas import *


device = "cuda" if torch.cuda.is_available() else "cpu"

/opt/conda/envs/smallcap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_data(tokenizer, max_length, features_dir, annotations_path, experiments_dir, encoder_name, decoder_name, attention_size, train_decoder,
     disable_rag, k, retrieval_encoder, captions_path, template_path, n_epochs, lr, batch_size, gradient_steps):

    data = load_data_for_training(annotations_path, captions_path)
    train_df = pd.DataFrame(data['train'])

    train_dataset = TrainDataset(
                           df=train_df,
                           features_path=os.path.join(features_dir,'train.hdf5'),
                           tokenizer=tokenizer,
                           rag=not disable_rag,
                           template_path=template_path,
                           k=k,
                           max_target_length=max_length)

    return train_dataset

In [7]:
def get_model_and_auxiliaries():

    
    # load feature extractor
    feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")

    # load and configure tokenizer
    tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
    tokenizer.pad_token = '!'
    tokenizer.eos_token = '.'

    # load model
    AutoConfig.register("this_opt", ThisOPTConfig)
    AutoModel.register(ThisOPTConfig, ThisOPTForCausalLM)
    AutoModelForCausalLM.register(ThisOPTConfig, ThisOPTForCausalLM)
    AutoConfig.register("smallcap", SmallCapConfig)
    AutoModel.register(SmallCapConfig, SmallCap)
    model = AutoModel.from_pretrained("Yova/SmallCap7M")
    model = model.to(device)
    
    # count trainable parameters
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    num_trainable_params = sum([np.prod(p.size()) for p in model_parameters])
    print('Training a model with {} trainable parameters.'.format(num_trainable_params))

    return model, tokenizer, feature_extractor

In [4]:
def get_model_and_auxiliaries():

    
    # load feature extractor
    feature_extractor = CLIPFeatureExtractor.from_pretrained("openai/clip-vit-base-patch32")

    # load and configure tokenizer
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = '!'
    tokenizer.eos_token = '.'

    # load model
    AutoConfig.register("this_gpt2", ThisGPT2Config)
    AutoModel.register(ThisGPT2Config, ThisGPT2LMHeadModel)
    AutoModelForCausalLM.register(ThisGPT2Config, ThisGPT2LMHeadModel)
    AutoConfig.register("smallcap", SmallCapConfig)
    AutoModel.register(SmallCapConfig, SmallCap)
    model = AutoModel.from_pretrained("Yova/SmallCap7M")
    model = model.to(device)
    
    # count trainable parameters
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    num_trainable_params = sum([np.prod(p.size()) for p in model_parameters])
    print('Training a model with {} trainable parameters.'.format(num_trainable_params))

    return model, tokenizer, feature_extractor

In [8]:
def main(features_dir, annotations_path, experiments_dir, encoder_name, decoder_name, attention_size, train_decoder,
     disable_rag, k, retrieval_encoder, captions_path, template_path, n_epochs, lr, batch_size, gradient_steps):

    model, tokenizer, feature_extractor = get_model_and_auxiliaries()
    train_dataset = get_data(tokenizer, model.config.max_length, features_dir, annotations_path, experiments_dir, \
                            encoder_name, decoder_name, attention_size, train_decoder, disable_rag, k, retrieval_encoder, \
                            captions_path, template_path, n_epochs, lr, batch_size, gradient_steps)

    model_type = 'norag' 

    output_dir = '{}_{}M_{}'.format(model_type, attention_size, decoder_name)
    output_dir = os.path.join(experiments_dir, output_dir)
    
    training_args = Seq2SeqTrainingArguments(
        num_train_epochs=n_epochs, 
        per_device_train_batch_size=batch_size, 
        gradient_accumulation_steps=gradient_steps,
        learning_rate = lr,
        fp16=True,
        save_strategy="epoch",
        save_total_limit=n_epochs, 
        logging_strategy="epoch", 
        output_dir=output_dir, 
        overwrite_output_dir=True, 
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        data_collator=default_data_collator, 
        train_dataset=train_dataset,
        tokenizer=feature_extractor,
    )

    trainer.train()

In [9]:
# Define arguments

#Directory where cached input image features are stored
features_dir =  'fine-tuning/features/'

#JSON file with annotations
annotations_path =  'fine-tuning/datastore/index.json'  

#Directory where trained models will be saved
experiments_dir =  "experiments/"  

encoder_name = "openai/clip-vit-base-patch32"  #Encoder name as found of HuggingFace or stored locally
decoder_name = "facebook/opt-350m"             #Decoder name as found of HuggingFace or stored locally
attention_size = 7                             #Number of parameters in the cross attention {28, 14, 7, 3.5, 1.75}
train_decoder = False                          #Whether to train the decoder in addition to the attention

disable_rag = False                            #Disable retrieval augmentation
k = 4                                          #Number of retrieved captions to use in prefix
retrieval_encoder = "RN50x64"                  #Visual encoder used for retieving captions

#JSON file with retrieved captions
captions_path = 'fine-tuning/data/retrieved_caps_resnet50x64.json'      

#TXT file with template
template_path = "src/template.txt"             

n_epochs = 8                                   #Number of training epochs
lr = 1e-4                                      #Learning rate
batch_size = 1                                 #Batch size
gradient_steps = 1                             #Number of gradient accumulation steps

# Call the main function with defined arguments
main(features_dir, annotations_path, experiments_dir, encoder_name, decoder_name, attention_size, train_decoder,
     disable_rag, k, retrieval_encoder, captions_path, template_path, n_epochs, lr, batch_size, gradient_steps)
     

loading feature extractor configuration file https://huggingface.co/openai/clip-vit-base-patch32/resolve/main/preprocessor_config.json from cache at /home/ec2-user/.cache/huggingface/transformers/70a7f0560266197c6c30797348b0d05575ccc9efe7b0025815d2f9800d223f7a.cc322732a6b876aad7dc614c3b22104fcc07458dc2df81953cff99baed365f9a
Feature extractor CLIPFeatureExtractor {
  "crop_size": 224,
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "CLIPFeatureExtractor",
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "resample": 3,
  "size": 224
}

loading configuration file https://huggingface.co/facebook/opt-350m/resolve/main/config.json from cache at /home/ec2-user/.cache/huggingface/transformers/d607f52c6959c98fd5116288e3118e4b536b5336e14e43b9893f467247c51903.1df7a93756b24a5859673be20e064b5c30771c0c8ca9e6a56ff30daa2885580b
Model co

Training a model with 257605632 trainable parameters.


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
/opt/conda/envs/smallcap/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11565
  Num Epochs = 8
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 92520
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "tr

Step,Training Loss
11565,2.197700
23130,1.388600
34695,1.042400
46260,0.808900
57825,0.646400
69390,0.528900
80955,0.437100
92520,0.365500


Saving model checkpoint to experiments/norag_7M_facebook/opt-350m/checkpoint-11565
Configuration saved in experiments/norag_7M_facebook/opt-350m/checkpoint-11565/config.json
Model weights saved in experiments/norag_7M_facebook/opt-350m/checkpoint-11565/pytorch_model.bin
Feature extractor saved in experiments/norag_7M_facebook/opt-350m/checkpoint-11565/preprocessor_config.json
Deleting older checkpoint [experiments/norag_7M_facebook/opt-350m/checkpoint-11565] due to args.save_total_limit
Saving model checkpoint to experiments/norag_7M_facebook/opt-350m/checkpoint-23130
Configuration saved in experiments/norag_7M_facebook/opt-350m/checkpoint-23130/config.json
Model weights saved in experiments/norag_7M_facebook/opt-350m/checkpoint-23130/pytorch_model.bin
Feature extractor saved in experiments/norag_7M_facebook/opt-350m/checkpoint-23130/preprocessor_config.json
Saving model checkpoint to experiments/norag_7M_facebook/opt-350m/checkpoint-34695
Configuration saved in experiments/norag_7M_fa

wandb: Waiting for W&B process to finish... (success).
